In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
from keras.models import load_model
import pickle as pkl
from data_preprocessing.image_processing import FeatureExtractor
from commons.utils import load_pickle_file
from keras.preprocessing.sequence import pad_sequences
from numpy import argmax
from models.model_eval import BeamSearch, get_bleu_score
from IPython.core.display import Image, display

In [ ]:
# Change the path according to where you stored each component here
captions = load_pickle_file('data/preprocessed/captions.pkl.test')
features = load_pickle_file('data/preprocessed/Xception_features.pkl.test')
tokenizer = load_pickle_file('data/preprocessed/tokenizer.pkl')
model = load_model('xception.epoch_04-loss_3.15.hdf5')

In [ ]:
def evaluate_model(model, captions, features, tokenizer, max_iter=50):
    y_true = []
    y_pred = []
    start = len('startseq')
    end = -len('endseq')
    beam_search = BeamSearch(model, tokenizer)
    i = 0
    for img_id, caption_list in captions.items():
        image_path = "data/Flicker8k_Dataset/" + img_id + ".jpg"
        display(Image(image_path, width=500))
        y, proba = beam_search.search(features[img_id])
        print("{} ({:.2E})\n".format(y, proba))
        refs = [c[start:end].split() for c in caption_list]
        y_true.append(refs)
        y_pred.append(y.split())
        i+=1
        if i >= max_iter:
            break
    get_bleu_score(y_true, y_pred)

In [ ]:
evaluate_model(model, captions, features, tokenizer)